In [372]:
import pandas as pd
import regex as re 
import numpy as np

cottages_data = pd.read_csv('results.csv')

pd.set_option('mode.chained_assignment', None)

cottages_data

,developer_name,oblast,project_area,project_name,project_price_uah,title,town,type_and_count
0,NaN,NaN,NaN,NaN,NaN,\n Таунхаус HIGH HILLS\n ...,Дніпро,"<table class=""CottageConstruction-table"">\n ..."
1,ЗахБудКом,NaN,NaN,NaN,NaN,\n Таунхауси Hodillywood 12.0\n...,Чернівці,"<table class=""CottageConstruction-table"">\n ..."
2,ЗахБудКом,Чернівецька область,"<div class=""CottagePrices-cell -area"">102.72 м...","<div class=""CottagePrices-cell -name"">Тип В</d...","<div data-currency=""uah"" class="""">\n ...",\n Таунхауси Hodillywood 7.0\n ...,с. Годилів,"<table class=""CottageConstruction-table"">\n ..."
3,NaN,Івано-Франківська область,"<div class=""CottagePrices-cell -area"">135 м²</...","<div class=""CottagePrices-cell -name"">Проєкт 1...","<div data-currency=""uah"" class="""">\n ...","\n вул. Вовчинецька, 38\n ...",с. Вовчинець,"<table class=""CottageConstruction-table"">\n ..."
...,...,...,...,...,...,...,...,...
859,Добробут Львів,NaN,"<div class=""CottagePrices-cell -area"">154 м²</...","<div class=""CottagePrices-cell -name"">Проєкт 2...","<div data-currency=""uah"" class="""">\n ...",\n вул. Роксоляни\n,Львів,"<table class=""CottageConstruction-table"">\n ..."
860,Phoenix Invest Group,Львівська область,"<div class=""CottagePrices-cell -area"">52 м²</d...","<div class=""CottagePrices-cell -name"">Проєкт 1...","<div data-currency=""uah"" class="""">\n ...",\n КМ PHOENIX MEDICAL RESORT\n ...,смт Східниця,"<table class=""CottageConstruction-table"">\n ..."
861,NaN,Львівська область,"<div class=""CottagePrices-cell -area"">174.12 м...","<div class=""CottagePrices-cell -name"">Бойківщи...","<div data-currency=""uah"" class="""">\n ...",\n КМ Золота Підкова\n,с. Давидів,"<table class=""CottageConstruction-table"">\n ..."
862,NaN,Львівська область,"<div class=""CottagePrices-cell -area"">93 м²</d...","<div class=""CottagePrices-cell -name"">Тип 1</d...","<div data-currency=""uah"" class="""">\n ...",\n КМ Панич 3.0\n,с. Сокільники,"<table class=""CottageConstruction-table"">\n ..."


 Data processing

In [373]:
cottages_data['title'] = cottages_data['title'].str.split().apply(' '.join)

Form lists of project prices, areas and names

In [374]:
cottages_data['project_price_uah'] = cottages_data['project_price_uah'].str.split()
for i in range(len(cottages_data['project_price_uah'])):
    if type(cottages_data['project_name'][i]) is not float:
        num = []
        for j in cottages_data['project_price_uah'][i]:
            try:
                float(j)
                if float(j) > 100 :
                    num.append(float(j)/1000)
                else: 
                    num.append(float(j))
            except:
                pass
        cottages_data['project_price_uah'][i] = num

In [375]:
cottages_data['project_area'] = cottages_data['project_area'].str.split(r'<|>| ')
for i in range(len(cottages_data['project_area'])):
    if type(cottages_data['project_area'][i]) is not float:
        num = []
        for j in cottages_data['project_area'][i]:
            try:
                num.append(float(j))
            except:
                pass
        cottages_data['project_area'][i] = num

In [376]:
cottages_data['project_name'] = cottages_data['project_name'].str.split(r'<div class="CottagePrices-cell -name">|</div>')
for i in range(len(cottages_data['project_name'])):
    if type(cottages_data['project_name'][i]) is not float:
        
# also cut the list, throwing out the names of not actual projects

        cottages_data['project_name'][i] = [j for j in cottages_data['project_name'][i] if len(j)>2][:len(cottages_data['project_price_uah'][i])]

Exist several variant to breakdown of type_and_count column. In this column information about building type how many are in projects, how many are under construction, how already been built. 
I choose option where list of types of buildings in one column and in the following the number by section.

In [377]:
cottages_data['type_and_count'] = cottages_data['type_and_count'].str.split()

cottages_data['in_project'] = 0
cottages_data['in_construction_stage'] = 0
cottages_data['builted'] = 0
cottages_data['types'] = ''

In [378]:
for i in range(len(cottages_data['type_and_count'])):
    if type(cottages_data['type_and_count'][i]) is not float:
        ls = cottages_data['type_and_count'][i][9:-2]
        temp_list = [] 
        for j in ls:
            if len(j) > 9:
                temp_list.append(j[4:-5])
        temp_list.remove('Вільні') if 'Вільні' in temp_list else None
        temp_list.remove('Забудовуються') if 'Забудовуються' in temp_list else None
        temp_list.remove('Забудовані') if 'Забудовані' in temp_list else None
        if 'Ділянки' in temp_list:
            temp_list = temp_list[:-4]
        type_list = temp_list[::4]
        project_list = temp_list[1::4]
        construction_list = temp_list[2::4]
        built_list = temp_list[3::4]
        #print(temp_list, i)
        cottages_data['in_project'][i] = sum(map(int, project_list))/len(project_list) if len(project_list) else 0
        cottages_data['in_construction_stage'][i] = sum(map(int, construction_list))/len(construction_list) if len(construction_list) else 0
        cottages_data['builted'][i] = sum(map(int, built_list))/len(built_list) if len(built_list) else 0
        cottages_data['types'][i] = type_list

In [379]:
cottages_data['total_count'] = cottages_data['in_project'] + cottages_data['in_construction_stage'] + cottages_data['builted']

In [197]:
# cottages_data = cottages_data.dropna(subset=['type_and_count'])

Look at average prices and areas for projects

In [380]:
cottages_data['average_project_price_uah'] = np.nan 
cottages_data['average_project_area'] = np.nan 

for i in range(len(cottages_data['project_price_uah'])):
    if type(cottages_data['project_name'][i]) is not float:
        if len(cottages_data['project_price_uah'][i]):
            price = cottages_data['project_price_uah'][i] 
            area = cottages_data['project_area'][i] 
            cottages_data['average_project_price_uah'][i] = sum(map(float, price))/len(price)
            cottages_data['average_project_area'][i] = sum(map(float, area))/len(area)

Data Analysis

Is no sense in the data in which the total_count = 0

In [381]:
cottages = cottages_data[cottages_data['total_count'] > 0] 

In [382]:
cottages = cottages.drop(columns=['type_and_count'])

In [383]:
cottages

,developer_name,oblast,project_area,project_name,project_price_uah,title,town,in_project,in_construction_stage,builted,types,total_count,average_project_price_uah,average_project_area
2,ЗахБудКом,Чернівецька область,[102.72],[Тип В],[2.89],Таунхауси Hodillywood 7.0,с. Годилів,0.0,0.0,12.0,[Таунхауси],12.0,2.890,102.72
3,NaN,Івано-Франківська область,"[135.0, 135.0, 135.0]","[Проєкт 1, Проєкт 1, Проєкт 1]","[2.27, 2.39, 2.27]","вул. Вовчинецька, 38",с. Вовчинець,0.0,5.0,5.0,[Таунхауси],10.0,2.310,135.00
5,Будівельний Альянс Груп,NaN,"[137.11, 133.49]","[Тип 3.1, Тип 2.2]","[3.2, 3.35]",Таунхауси „Мікрорайон Дендропарковий“,Хмельницький,0.0,20.0,43.0,[Таунхауси],63.0,3.275,135.30
6,Мегаінвестбуд,NaN,NaN,NaN,NaN,Таунхаус Shatilovka Hillside,Харків,0.0,0.0,21.0,[Таунхауси],21.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,Добробут Львів,NaN,"[154.0, 178.0]","[Проєкт 2, Проєкт 4]","[4.45, 5.14]",вул. Роксоляни,Львів,0.0,0.0,6.0,[Таунхауси],6.0,4.795,166.00
860,Phoenix Invest Group,Львівська область,"[52.0, 116.0, 52.0, 116.0, 52.0, 116.0, 52.0, ...","[Проєкт 1, Проєкт 2 , Проєкт 1, Проєкт 2 , Про...","[3.42, 6.51, 3.42, 6.51, 3.42, 6.51, 3.42, 6.51]",КМ PHOENIX MEDICAL RESORT,смт Східниця,146.0,10.0,0.0,[Котеджі],156.0,4.965,84.00
861,NaN,Львівська область,"[174.12, 216.5]","[Бойківщина, Лемківщина]","[3.39, 3.77]",КМ Золота Підкова,с. Давидів,49.0,3.0,10.0,[Котеджі],62.0,3.580,195.31
862,NaN,Львівська область,"[93.0, 110.0]","[Тип 1, Тип 2]","[4.08, 4.83]",КМ Панич 3.0,с. Сокільники,0.0,6.0,0.0,[Таунхауси],6.0,4.455,101.50


Let's look at the median number of houses in one cottage town, the median cost of house project (in millions of UAH), the median area of house. By all country.

In [384]:
cottages[['total_count', 'average_project_area', 'average_project_price_uah']].median() 

total_count                   13.0
average_project_area         125.1
average_project_price_uah      3.6
dtype: float64

Let's look at the 90th percentile for the area, price (in millions of UAH), and total count. By all country.

In [385]:
cottages[['total_count', 'average_project_area', 'average_project_price_uah']].quantile (0.9)

total_count                   78.333333
average_project_area         220.539484
average_project_price_uah     10.440900
Name: 0.9, dtype: float64

You can also add 'in_project', 'in_construction_stage', 'builted' columns to show statistic by project stages

In [386]:
pd.set_option('display.max_rows', 8) # count of show records

In [387]:
columns_list = ['title', 'types', 'average_project_price_uah', 'average_project_area', 'in_project', 'in_construction_stage', 'builted', 'total_count', 'developer_name']

You can also add 'project_area', 'project_name', 'project_price_uah' columns to columns_list to show more about cottage projects

In [388]:
def region(city, oblast):
    return cottages[(cottages['oblast'] == oblast) | (cottages['town'] == city)]

In [389]:
def statistic(df):
    m = df[['total_count', 'average_project_area', 'average_project_price_uah']].median()
    q = df[['total_count', 'average_project_area', 'average_project_price_uah']].quantile (0.9)
    print(f"Median values \n houses: {round(m[0],2)}, area: {round(m[1],2)}, price (M uah): {round(m[2],2)}")
    print(f"Quantiles \n houses: {round(q[0],2)}, area: {round(q[1],2)}, price (M uah): {round(q[2],2)}")

Next sections separate by metropolitan area

Each section has table, after it median values and percentages.

Kyiv region

In [390]:
cottages_kyiv_oblast = cottages[cottages['town'] == 'Київська область']
cottages_kyiv_oblast[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
126,КМ Тарасівка,"[Котеджі, Таунхауси]",2.060714,104.785714,37.0,3.5,44.5,85.0,БК Контакт
131,КМ Нова Ходосівка,[Котеджі],12.453684,271.657895,0.0,4.0,33.0,37.0,NaN
133,КМ VILLAGGO,"[Котеджі, Таунхауси]",6.641176,128.315294,0.0,17.0,90.5,107.5,NaN
136,Таунхауси Freya,[Таунхауси],3.044000,98.800000,51.0,24.0,0.0,75.0,NaN
...,...,...,...,...,...,...,...,...,...
756,КМ Хуторок,"[Котеджі, Таунхауси]",NaN,NaN,0.0,0.0,129.0,129.0,NaN
757,КМ Еліт Чайка,[Котеджі],7.375000,165.000000,15.0,4.0,0.0,19.0,NaN
759,КМ Дачная деревня. Вышеград,[Котеджі],NaN,NaN,0.0,0.0,9.0,9.0,NaN
761,КМ Дачная деревня. Червона Слобода,[Котеджі],NaN,NaN,8.0,0.0,9.0,17.0,NaN


In [391]:
statistic(cottages_kyiv_oblast)

Median values 
 houses: 15.0, area: 120.0, price (M uah): 3.55
Quantiles 
 houses: 95.9, area: 221.74, price (M uah): 11.34


Kyiv city

In [392]:
cottages_kyiv = cottages[cottages['town'] == 'Київ']
cottages_kyiv[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
132,Таунхауси „New Smart 6“,"[Квадрекси, Таунхауси]",2.873333,72.0,0.0,0.0,17.0,17.0,NaN
223,Таунхауси Park Golosievo,[Таунхауси],9.630000,194.0,0.0,4.0,0.0,4.0,NaN
227,КМ Набережний,[Котеджі],10.395000,226.0,0.0,0.0,8.0,8.0,NaN
307,КМ BritanHouse,[Котеджі],9.243333,227.0,0.0,9.0,3.0,12.0,NaN
...,...,...,...,...,...,...,...,...,...
718,КМ Dream Wood,"[Котеджі, Дуплекси]",NaN,NaN,0.0,0.0,11.0,11.0,Global Development (Ірпінь)
729,Таунхауси San Francisco,[Таунхауси],NaN,NaN,4.0,0.0,0.0,4.0,NaN
744,Таунхауси City Residence,[Таунхауси],NaN,NaN,0.0,4.0,0.0,4.0,NaN
746,КМ Чапаєвка,"[Котеджі, Дуплекси]",NaN,NaN,0.0,0.5,2.0,2.5,NaN


In [393]:
statistic(cottages_kyiv)

Median values 
 houses: 8.0, area: 178.83, price (M uah): 7.22
Quantiles 
 houses: 18.2, area: 248.2, price (M uah): 16.5


Lviv region

In [394]:
cottages_lviv = region('Львів', 'Львівська область')
cottages_lviv[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
127,Таунхаус RENAISSANCE,[Таунхауси],7.370,167.925,0.0,8.0,0.0,8.0,NaN
128,КМ Red Wood,[Таунхауси],3.760,108.530,0.0,6.0,3.0,9.0,NaN
129,Таунхауси „Rubicon Cottages“,[Таунхауси],4.430,139.400,0.0,0.0,14.0,14.0,NaN
130,КМ Timberland,[Котеджі],6.350,150.000,0.0,1.0,8.0,9.0,Forward Group
...,...,...,...,...,...,...,...,...,...
859,вул. Роксоляни,[Таунхауси],4.795,166.000,0.0,0.0,6.0,6.0,Добробут Львів
860,КМ PHOENIX MEDICAL RESORT,[Котеджі],4.965,84.000,146.0,10.0,0.0,156.0,Phoenix Invest Group
861,КМ Золота Підкова,[Котеджі],3.580,195.310,49.0,3.0,10.0,62.0,NaN
862,КМ Панич 3.0,[Таунхауси],4.455,101.500,0.0,6.0,0.0,6.0,NaN


In [395]:
statistic(cottages_lviv)

Median values 
 houses: 10.0, area: 147.61, price (M uah): 4.43
Quantiles 
 houses: 50.0, area: 199.18, price (M uah): 7.01


Kharkiv region

In [396]:
cottages_kh = region('Харків', 'Харківська область')
cottages_kh[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
6,Таунхаус Shatilovka Hillside,[Таунхауси],NaN,NaN,0.0,0.0,21.0,21.0,Мегаінвестбуд
7,КМ Графський,[Котеджі],NaN,NaN,0.0,0.0,52.0,52.0,NaN
8,КМ SKANDINAVSKY,"[Котеджі, Таунхауси]",NaN,NaN,13.0,0.0,11.5,24.5,NaN
9,КМ Добрі сусіди,[Котеджі],NaN,NaN,0.0,0.0,11.0,11.0,NaN
...,...,...,...,...,...,...,...,...,...
29,Таунхауси в П'ятихатках,[Таунхауси],6.313333,348.0,500.0,34.0,140.0,674.0,NaN
30,Таунхаус „Холодногірський“,[Таунхауси],3.723333,175.0,0.0,0.0,22.0,22.0,STROIM group
803,"просп. Жуковського, 7",[Таунхауси],NaN,NaN,0.0,0.0,7.0,7.0,NaN
806,КМ Молодежная резиденция,"[Котеджі, Таунхауси]",NaN,NaN,0.0,0.0,18.0,18.0,NaN


In [367]:
statistic(cottages_kh)

Median values 
 houses: 23.25, area: 120.64, price (M uah): 5.48
Quantiles 
 houses: 90.0, area: 321.73, price (M uah): 13.99


Одеська область

In [397]:
cottages_od = region('Одеса', 'Одеська область')
cottages_od[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
348,"вул. Таллінська, 6",[Таунхауси],NaN,NaN,0.0,4.0,0.0,4.0,Південна будівна
350,Таунхауси „САДИБА“,[Таунхауси],NaN,NaN,0.0,25.0,0.0,25.0,NaN
351,КМ Будинки на Ковалевського,[Котеджі],7.05,183.0,0.0,0.0,4.0,4.0,NaN
352,КМ Harmony house,[Котеджі],8.55,222.0,0.0,0.0,3.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...
794,Таунхауси „На Центральній“,[Таунхауси],NaN,NaN,0.0,0.0,24.0,24.0,NaN
796,Таунхауси „Голландія“,[Таунхауси],NaN,NaN,0.0,0.0,10.0,10.0,NaN
798,Iner Home,[Таунхауси],NaN,NaN,23.0,0.0,0.0,23.0,NaN
799,КМ Жемчужный,[Котеджі],NaN,NaN,0.0,0.0,2.0,2.0,NaN


In [365]:
statistic(cottages_od)

Median values 
 houses: 13.33, area: 130.88, price (M uah): 4.81
Quantiles 
 houses: 146.4, area: 299.5, price (M uah): 20.79


Вінницька область

In [398]:
cottages_vn = region('Вінниця', 'Вінницька область')
cottages_vn[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
178,Таунхаус „Amsterdom“,[Таунхауси],NaN,NaN,5.0,0.0,0.0,5.0,NaN
185,Таунхауси „Sunny Village“,[Таунхауси],NaN,NaN,0.0,0.0,11.0,11.0,NaN
187,Таунхауси 3 EON,[Таунхауси],NaN,NaN,6.0,8.0,0.0,14.0,NaN
188,КМ Green city,"[Котеджі, Таунхауси]",NaN,NaN,0.0,0.0,27.0,27.0,NaN
...,...,...,...,...,...,...,...,...,...
207,КМ Екологічне,[Таунхауси],2.89,135.000000,0.0,0.0,22.0,22.0,NaN
209,Таунхауси La Vie,[Таунхауси],2.62,98.740000,0.0,0.0,6.0,6.0,NaN
210,Таунхауси „Dream Town“,[Таунхауси],7.96,148.833333,0.0,0.0,26.0,26.0,Квадратні Метри М²
211,пров. Срібний,[Таунхауси],2.39,115.000000,0.0,0.0,10.0,10.0,NaN


In [363]:
statistic(cottages_vn)

Median values 
 houses: 10.5, area: 122.5, price (M uah): 2.76
Quantiles 
 houses: 25.6, area: 144.05, price (M uah): 5.08


Дніпропетровська область

In [399]:
cottages_dn = region('Дніпро', 'Дніпропетровська область')
cottages_dn[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
172,КМ Шишкино,"[Котеджі, Дуплекси, Таунхауси]",NaN,NaN,0.666667,0.0,10.666667,11.333333,NaN
174,Таунхаус „Садовий“,[Таунхауси],3.570000,123.500000,4.000000,4.0,0.000000,8.000000,NaN
175,КМ Hygge Village,"[Котеджі, Дуплекси]",NaN,NaN,1.000000,0.5,1.000000,2.500000,NaN
176,КМ Fill House,[Котеджі],3.420000,118.300000,0.000000,11.0,1.000000,12.000000,NaN
...,...,...,...,...,...,...,...,...,...
197,Таунхауси Living Place,[Таунхауси],3.560000,82.533333,0.000000,0.0,10.000000,10.000000,NaN
199,КМ ЛісоПаркове,[Таунхауси],3.130000,125.200000,15.000000,0.0,18.000000,33.000000,NaN
200,КМ Sun Coast,[Котеджі],1.488300,22.800000,0.000000,0.0,2.000000,2.000000,NaN
204,КМ Барвіта,"[Котеджі, Таунхауси]",1.317571,63.857143,10.000000,0.0,13.000000,23.000000,NaN


In [360]:
statistic(cottages_dn)

Median values 
 houses: 11.33, area: 118.3, price (M uah): 3.42
Quantiles 
 houses: 43.6, area: 153.24, price (M uah): 7.25


Житомирська область

In [400]:
cottages_gt = region('Житомир', 'Житомирська область')
cottages_gt[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
213,КМ Княжий Хутор,[Котеджі],1.287182,99.136364,114.0,0.0,38.0,152.0,NaN
214,Таунхауси „Європейський квартал“,[Таунхауси],4.140000,153.455000,6.0,8.0,0.0,14.0,NaN
216,Квадрекс Ihouse Quadro,[Квадрекси],2.890000,125.000000,0.0,4.0,0.0,4.0,Ihouse Development
217,Premium Space,"[Дуплекси, Таунхауси]",3.653571,113.557143,37.0,12.0,0.0,49.0,ГК «Premium Space»
219,"пров. Кам’яний узвіз, 12, 12а",[Таунхауси],NaN,NaN,0.0,0.0,14.0,14.0,NaN
221,Таунхауси „Royal family club“,[Таунхауси],2.985000,102.000000,134.0,32.0,16.0,182.0,NaN


In [358]:
statistic(cottages_gt)

Median values 
 houses: 31.5, area: 113.56, price (M uah): 2.99
Quantiles 
 houses: 167.0, area: 142.07, price (M uah): 3.95


Івано-Франківська область

In [401]:
cottages_if = region('Івано-Франківськ', 'Івано-Франківська область')
cottages_if[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
3,"вул. Вовчинецька, 38",[Таунхауси],2.310000,135.000000,0.0,5.0,5.0,10.0,NaN
51,Таунхауси SmartBud,[Таунхауси],2.755000,140.000000,0.0,0.0,4.0,4.0,NaN
52,Таунхауси „Sttk #9“,[Таунхауси],2.660000,140.000000,0.0,4.0,0.0,4.0,Sttk Development
53,КМ Yablunytsia,[Котеджі],7.905000,128.350000,0.0,9.0,1.0,10.0,UBH
...,...,...,...,...,...,...,...,...,...
784,КМ Eco Land,[Котеджі],7.007143,165.428571,6.0,0.0,1.0,7.0,NaN
785,Дуплекси Atlant House,[Дуплекси],6.160000,197.000000,0.0,0.0,8.0,8.0,NaN
786,КМ Hygge Town,[Таунхауси],NaN,NaN,0.0,10.0,0.0,10.0,NaN
790,Таунхауси „Sttk #8“,[Таунхауси],2.270000,110.000000,0.0,5.0,0.0,5.0,Sttk Development


In [356]:
statistic(cottages_if)

Median values 
 houses: 10.5, area: 130.0, price (M uah): 6.16
Quantiles 
 houses: 33.0, area: 173.0, price (M uah): 14.07


Запорізька область

In [353]:
cottages_zp = region('Запоріжжя', 'Запорізька область')
cottages_zp[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name


In [354]:
statistic(cottages_zp)

Median values 
 houses: nan, area: nan, price (M uah): nan
Quantiles 
 houses: nan, area: nan, price (M uah): nan


Тернопільська область

In [351]:
cottages_tr = region('Тернопіль', 'Тернопільська область')
cottages_tr[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
97,вул. Лісова,[Котеджі],NaN,NaN,0.0,6.0,0.0,6.0,NaN
98,КМ Кленовий гай,[Котеджі],2.70,215.0,0.0,0.0,200.0,200.0,Добробуд
102,вул. Козацька,[Котеджі],NaN,NaN,0.0,0.0,20.0,20.0,NaN
103,КМ Victoria Garden,[Дуплекси],1.83,92.5,0.0,0.0,110.0,110.0,NaN
104,КМ ZLAGODA CITY,[Таунхауси],2.77,177.5,0.0,7.0,7.0,14.0,NaN


In [352]:
statistic(cottages_tr)

Median values 
 houses: 20.0, area: 177.5, price (M uah): 2.7
Quantiles 
 houses: 164.0, area: 207.5, price (M uah): 2.76


Волинська область

In [402]:
cottages_vl = region('Луцьк', 'Волинська область')
cottages_vl[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
182,КМ Сімейний квартал,[Котеджі],NaN,NaN,0.0,0.0,36.0,36.0,NaN
192,Дуплекси OPTIMUS,[Дуплекси],2.690,111.0000,0.0,4.0,0.0,4.0,NaN
193,Таунхауси „ЗАРІЧЧЯ“,[Таунхауси],3.805,158.5875,0.0,0.0,8.0,8.0,NaN
194,Таунхауси „Мій Дім“,[Таунхауси],2.120,115.0000,0.0,0.0,8.0,8.0,NaN
...,...,...,...,...,...,...,...,...,...
203,КМ Династія,[Таунхауси],2.990,147.0000,0.0,0.0,68.0,68.0,Луцьксантехмонтаж №536
206,Таунхауси Green Home,[Таунхауси],1.930,92.0000,12.0,0.0,12.0,24.0,Орбіта
208,КМ Струмочок 2,"[Котеджі, Таунхауси]",3.580,157.0000,0.0,0.0,37.5,37.5,NaN
212,КМ Octava,[Котеджі],5.390,142.0000,49.0,14.0,0.0,63.0,NaN


In [350]:
statistic(cottages_vl)

Median values 
 houses: 19.25, area: 142.0, price (M uah): 2.99
Quantiles 
 houses: 63.5, area: 164.07, price (M uah): 5.45


Хмельницька область

In [403]:
cottages_hm = region('Хмельницький', 'Хмельницька область')
cottages_hm[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
5,Таунхауси „Мікрорайон Дендропарковий“,[Таунхауси],3.275000,135.300000,0.0,20.0,43.0,63.0,Будівельний Альянс Груп
31,КМ Європейський-2,[Котеджі],NaN,NaN,218.0,0.0,0.0,218.0,Dita
32,Таунхауси „Андора“,[Таунхауси],3.005000,104.000000,0.0,0.0,30.0,30.0,NaN
33,Таунхауси Dream Town,[Таунхауси],NaN,NaN,0.0,6.0,6.0,12.0,NaN
...,...,...,...,...,...,...,...,...,...
40,Таунхауси SMART & GREEN,[Таунхауси],3.416923,136.568462,0.0,15.0,18.0,33.0,Smart Green
41,"вул. Яворницького, 59/1",[Таунхауси],1.920000,80.000000,0.0,3.0,0.0,3.0,Будівельна компанія Sky
42,Таунхаус Містечко Північне“,[Таунхауси],NaN,NaN,22.0,0.0,10.0,32.0,NaN
44,КМ Eurovillage 2,[Таунхауси],2.320000,83.000000,0.0,44.0,0.0,44.0,NaN


In [348]:
statistic(cottages_hm)

Median values 
 houses: 32.0, area: 120.75, price (M uah): 2.97
Quantiles 
 houses: 61.2, area: 143.9, price (M uah): 3.43


Черкаська область

In [404]:
cottages_chs = region('Черкаси', 'Черкаська область')
cottages_chs[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
46,Таунхауси „Рівер Таун“,[Таунхауси],NaN,NaN,45.0,0.0,45.0,90.0,NaN
47,КМ Villa de Lux,[Котеджі],NaN,NaN,8.0,2.0,7.0,17.0,Строй time
48,КМ Dream Town,[Дуплекси],NaN,NaN,6.0,9.0,26.0,41.0,NaN
49,КМ Екохаус,[Таунхауси],1.680000,80.000000,0.0,62.0,0.0,62.0,NaN
50,Таунхаус „Солов'їний“,[Таунхауси],2.891667,95.006667,0.0,7.0,0.0,7.0,NaN


In [346]:
statistic(cottages_chs)

Median values 
 houses: 41.0, area: 87.5, price (M uah): 2.29
Quantiles 
 houses: 78.8, area: 93.51, price (M uah): 2.77


Рівненська область

In [405]:
cottages_rv = region('Рівне', 'Рівненська область')
cottages_rv[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
108,Таунхауси „Dream House“,[Таунхауси],NaN,NaN,12.0,0.0,12.0,24.0,NaN
109,Таунхаус Villa Italia,[Таунхауси],NaN,NaN,0.0,0.0,16.0,16.0,NaN
110,КМ Golden Hills,[Котеджі],NaN,NaN,0.0,0.0,76.0,76.0,NaN
111,Таунхауси „Княжий“,[Таунхауси],NaN,NaN,0.0,0.0,94.0,94.0,NaN
...,...,...,...,...,...,...,...,...,...
118,"вул. Київська, 135",[Квадрекси],2.160000,149.00,0.0,0.0,8.0,8.0,Optimus
119,"вул. Князя Острозького, 3б",[Таунхауси],2.460000,99.90,0.0,0.0,13.0,13.0,NaN
121,КМ Країна Мрій,"[Дуплекси, Таунхауси]",1.698889,106.26,0.0,28.5,52.0,80.5,NaN
122,Таунхаус «Гармонія-1»,[Таунхауси],1.890000,95.00,0.0,16.0,0.0,16.0,Optimus


In [344]:
statistic(cottages_rv)

Median values 
 houses: 16.0, area: 103.08, price (M uah): 2.14
Quantiles 
 houses: 79.6, area: 143.4, price (M uah): 2.86


Миколаївська область

In [340]:
cottages_my = region('Миколаїв', 'Миколаївська область')
cottages_my[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
215,КМ Княжичі,[Таунхауси],4.660,242.000,0.0,0.0,18.0,18.0,Констракшн ЛТД
218,"вул. Леваневців, 26",[Таунхауси],6.128,159.116,0.0,0.0,6.0,6.0,Antares City
220,КМ 6 Element,[Котеджі],6.350,203.000,0.0,0.0,6.0,6.0,Н-Інвест
222,"вул. Лікаря Ніколенка, 11",[Котеджі],4.430,185.000,0.0,3.0,3.0,6.0,NaN


In [341]:
statistic(cottages_my)

Median values 
 houses: 6.0, area: 194.0, price (M uah): 5.39
Quantiles 
 houses: 14.4, area: 230.3, price (M uah): 6.28


Полтавська область

In [337]:
cottages_pol = region('Полтава', 'Полтавська область')
cottages_pol[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
120,"вул. Переяславська, 4/2",[Таунхауси],3.570000,265.000000,0.0,0.0,6.00,6.00,Европабуд
123,КМ KOPYLY CLUB,[Таунхауси],3.493333,113.333333,0.0,31.0,0.00,31.00,ANGRO
124,КМ Ecopark,"[Котеджі, Дуплекси, Квадрекси, Таунхауси]",5.753333,143.333333,2.0,5.5,8.75,16.25,NaN


In [338]:
statistic(cottages_pol)

Median values 
 houses: 16.25, area: 143.33, price (M uah): 3.57
Quantiles 
 houses: 28.05, area: 240.67, price (M uah): 5.32


Чернівецька область

In [406]:
cottages_chv = region('Чернівці', 'Чернівецька область')
cottages_chv[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
2,Таунхауси Hodillywood 7.0,[Таунхауси],2.890000,102.720,0.0,0.0,12.0,12.0,ЗахБудКом
74,КМ VIVA-CHERNIVTSI,"[Котеджі, Таунхауси]",3.280000,128.000,0.0,4.5,1.5,6.0,NaN
75,Таунхауси Real House,[Таунхауси],4.720000,175.000,0.0,0.0,5.0,5.0,NaN
76,Таунхауси Hodillywood 2.0,[Таунхауси],2.416667,107.000,0.0,20.0,0.0,20.0,ЗахБудКом
...,...,...,...,...,...,...,...,...,...
83,Таунхауси Скай Таун,[Таунхауси],2.020000,110.700,0.0,3.0,0.0,3.0,City of Dreams
84,Таунхауси „Мальованка“,[Таунхауси],2.450000,96.225,26.0,64.0,40.0,130.0,City of Dreams
85,Таунхауси Emerald,[Таунхауси],3.930000,120.000,0.0,25.0,12.0,37.0,БК Водограй
86,КМ вул. Левицького (Кагарлицька),[Таунхауси],3.000000,120.000,0.0,0.0,5.0,5.0,Чернівці Моноліт


In [335]:
statistic(cottages_chv)

Median values 
 houses: 10.0, area: 120.0, price (M uah): 3.16
Quantiles 
 houses: 33.6, area: 181.16, price (M uah): 4.56


Кіровоградська область

In [333]:
cottages_ki = region('Кропивницький', 'Кіровоградська область')
cottages_ki[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
554,Таунхауси Forest House,[Таунхауси],3.6,138.35,0.0,0.0,8.0,8.0,NaN


In [332]:
statistic(cottages_ki)

Median values 
 houses: 8.0, area: 138.35, price (M uah): 3.6
Quantiles 
 houses: 8.0, area: 138.35, price (M uah): 3.6


Чернігівська область

In [329]:
cottages_chr = region('Чернігів', 'Чернігівська область')
cottages_chr[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
25,Таунхауси Eurovillage,[Таунхауси],3.505,129.5,0.0,85.0,0.0,85.0,Belin City


In [330]:
statistic(cottages_chr)

Median values 
 houses: 85.0, area: 129.5, price (M uah): 3.5
Quantiles 
 houses: 85.0, area: 129.5, price (M uah): 3.5


Сумська область

In [327]:
cottages_su = region('Суми', 'Сумська область')
cottages_su[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
72,КМ Титул,[Дуплекси],NaN,NaN,98.0,0.0,152.0,250.0,NaN
73,Таунхауси Town Park,[Таунхауси],2.89,124.05,0.0,0.0,11.0,11.0,NaN


In [328]:
statistic(cottages_su)

Median values 
 houses: 130.5, area: 124.05, price (M uah): 2.89
Quantiles 
 houses: 226.1, area: 124.05, price (M uah): 2.89


Закарпатська область

In [407]:
cottages_ug= region('Ужгород', 'Закарпатська область')
cottages_ug[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name
87,Таунхауси „Modena“,[Таунхауси],NaN,NaN,0.000000,0.000000,5.0,5.000000,NaN
88,Таунхаус „ДУЛОВА“,[Таунхауси],6.410000,238.00,0.000000,0.000000,12.0,12.000000,NaN
89,КМ ВДОМА,[Таунхауси],4.206667,162.60,0.000000,3.000000,8.0,11.000000,NaN
90,Таунхаус „ДЖЕРЕЛЬНА“,[Таунхауси],7.720000,167.00,0.000000,0.000000,6.0,6.000000,NaN
...,...,...,...,...,...,...,...,...,...
100,КМ HAPPY HOUSING,[Котеджі],3.850000,129.00,0.000000,6.000000,16.0,22.000000,NaN
101,КМ КРАСІЯ РЕЗІДЕНС,[Котеджі],2.045000,50.63,15.000000,0.000000,0.0,15.000000,РЕЛЬЕФ ЛТД
105,КМ SOHO,"[Котеджі, Дуплекси, Таунхауси]",3.288000,131.60,17.666667,6.666667,0.0,24.333333,NaN
106,КМ SHERWOOD,"[Дуплекси, Таунхауси]",7.840000,144.00,7.000000,12.000000,14.5,33.500000,NaN


In [326]:
statistic(cottages_ug)

Median values 
 houses: 12.0, area: 136.84, price (M uah): 3.9
Quantiles 
 houses: 29.83, area: 169.8, price (M uah): 7.68


Kherson region

In [323]:
cottages_h = region('Херсон', 'Херсонська область')
cottages_h[columns_list]

,title,types,average_project_price_uah,average_project_area,in_project,in_construction_stage,builted,total_count,developer_name


In [324]:
statistic(cottages_h)

Median values 
 houses: nan, area: nan, price (M uah): nan
Quantiles 
 houses: nan, area: nan, price (M uah): nan
